In [1]:
import pandas as pd
from google_play_scraper import reviews
from datetime import datetime

In [2]:
# --- Pengaturan ---
app_id = 'com.tokopedia.tkpd'

# Tentukan rentang waktu yang Anda inginkan (termasuk jam dan menit jika perlu)
start_date = datetime(2025, 9, 29)
end_date = datetime(2025, 11, 7)

all_reviews_list = []
continuation_token = None
# ------------------

print(f"Memulai scraping untuk ulasan dari {start_date} hingga {end_date}...")

# Loop akan terus berjalan sampai kita menghentikannya secara manual
while True:
    result, continuation_token = reviews(
        app_id,
        country='id',
        count=200,  # Ambil 200 ulasan per halaman
        continuation_token=continuation_token
    )

    # Tambahkan hasil ulasan yang baru didapat ke list utama
    all_reviews_list.extend(result)
    print(f"Total ulasan terkumpul: {len(all_reviews_list)}")
    
    # Cek jika tidak ada hasil lagi atau tidak ada token berikutnya
    if not continuation_token or not result:
        print("Tidak ada ulasan lebih lanjut. Proses scraping berhenti.")
        break
        
    # Ambil tanggal dari ulasan TERAKHIR di batch saat ini
    last_review_date = result[-1]['at']

    # Jika tanggal ulasan terakhir sudah lebih tua dari start_date kita,
    # kita bisa berhenti karena halaman-halaman berikutnya pasti lebih tua lagi.
    if last_review_date < start_date:
        print("Sudah mencapai batas tanggal mulai. Proses scraping berhenti.")
        break

# --- Proses Filtering (Dengan Perbaikan) ---
print("\nMemulai proses filtering data...")

df = pd.DataFrame(all_reviews_list)

# 1. Konversi kolom 'at' menjadi format datetime
df['at'] = pd.to_datetime(df['at'])

# 2. HAPUS INFORMASI TIMEZONE UNTUK MENSTANDARKAN (INI PERBAIKANNYA)
df['at'] = df['at'].dt.tz_localize(None)

# 3. Lakukan filtering akhir yang sekarang sudah akurat
filtered_df = df[
    (df['at'] >= start_date) & (df['at'] <= end_date)
]

print(f"Jumlah ulasan setelah difilter secara akurat: {len(filtered_df)}")

# Simpan hasil akhir ke CSV
if not filtered_df.empty:
    file_name = f'tokopedia_reviews_FIXED_{start_date.date()}_to_{end_date.date()}.csv'
    filtered_df.to_csv(file_name, index=False)
    print(f"File '{file_name}' berhasil disimpan.")
else:
    print("Tidak ada ulasan yang ditemukan pada rentang tanggal tersebut.")

Memulai scraping untuk ulasan dari 2025-09-29 00:00:00 hingga 2025-11-07 00:00:00...
Total ulasan terkumpul: 200
Total ulasan terkumpul: 400
Total ulasan terkumpul: 600
Total ulasan terkumpul: 800
Sudah mencapai batas tanggal mulai. Proses scraping berhenti.

Memulai proses filtering data...
Jumlah ulasan setelah difilter secara akurat: 713
File 'tokopedia_reviews_FIXED_2025-09-29_to_2025-11-07.csv' berhasil disimpan.


In [3]:
df = pd.read_csv(f'{file_name}')
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,319182d9-dc65-4176-b2e2-3de56dc7566c,anton wiyono,https://play-lh.googleusercontent.com/a/ACg8oc...,APLIKASINYA PARAH...BUAT SCROLL² KE BAWAH BBER...,1,0,3.336.0,2025-11-06 18:27:49,NaN,NaN,3.336.0
1,c93ffe76-93ce-4817-9dd3-e36f03a06e41,Gusti Pangestu,https://play-lh.googleusercontent.com/a-/ALV-U...,harus ada komplain langsung dari customer sama...,1,0,3.333.0,2025-11-06 18:07:01,NaN,NaN,3.333.0
2,a0a29883-be97-44b8-b6fd-d7f0cd080f1d,Ryan Zulkifli,https://play-lh.googleusercontent.com/a-/ALV-U...,App performance is getting worse,1,0,3.336.0,2025-11-06 17:38:23,NaN,NaN,3.336.0
3,a232c784-7471-4da8-9178-f4d44adcc7fd,War Street,https://play-lh.googleusercontent.com/a/ACg8oc...,no longer direct buy via laptop. forcing buyin...,1,0,NaN,2025-11-06 16:58:15,NaN,NaN,NaN
4,5a3354ab-e08c-441a-bb90-0610ad71f6a8,Rizky Dermawan,https://play-lh.googleusercontent.com/a-/ALV-U...,"Update ASAP woy, makin gaje app-nya loading pi...",1,0,3.337.1,2025-11-06 15:23:56,NaN,NaN,3.337.1
